In [13]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict,Annotated
from pydantic import BaseModel, Field
import operator

In [2]:
load_dotenv()

True

In [4]:
model=ChatOpenAI(model='gpt-4o-mini')

In [24]:
class eval_schema(BaseModel):
    feedback:str=Field(description="Detailed feedback for the essay")
    score:int=Field(description='score out of 10', ge=0, le=10)
    

In [25]:
structured_model=model.with_structured_output(eval_schema)

In [26]:
essay="""Artificial Intelligence (AI) has become one of the most transformative technologies of the 21st century. It is reshaping industries, economies, and societies across the world. India, being one of the fastest-growing economies with a strong talent base in technology, has been playing an important role in the global AI ecosystem. From government initiatives to private sector innovations, India’s contribution to AI is becoming more significant day by day.

1. Government Initiatives

The Government of India has recognized AI as a key driver for economic growth and social development. NITI Aayog launched the National Strategy for Artificial Intelligence, often called “AI for All”, with the aim of using AI in areas such as healthcare, agriculture, education, smart cities, and infrastructure. The government is also setting up centers of excellence and AI research hubs to encourage innovation and collaboration.

2. Research and Innovation

India is home to some of the best engineering institutions in the world, such as the IITs, IISc, and IIITs, which are actively conducting research in machine learning, deep learning, natural language processing, and robotics. Startups in cities like Bengaluru, Hyderabad, and Pune are focusing on AI-based solutions ranging from financial technology and healthcare diagnostics to agricultural optimization. This research ecosystem has made India one of the top countries in terms of AI research publications.

3. Private Sector and Startups

The private sector in India is heavily investing in AI. Major IT companies such as TCS, Infosys, and Wipro are using AI to improve services for clients worldwide. Global tech giants like Google, Microsoft, and Amazon have established AI research and development centers in India, recognizing the country’s pool of skilled engineers and data scientists. Indian startups are also playing a crucial role in areas like speech recognition in regional languages, AI-powered education platforms, and medical imaging solutions.

4. AI for Social Good

One of India’s unique strengths is its focus on applying AI to solve local problems. For example, AI is being used in agriculture to predict crop diseases and improve yield. In healthcare, AI-based diagnostic tools are helping doctors in rural areas provide better treatment. Educational platforms powered by AI are helping students in remote villages access quality learning material. Thus, AI in India is not just about business but also about inclusive growth.

5. Challenges Ahead

Despite these achievements, India faces challenges such as limited infrastructure, shortage of high-quality datasets, and the need for more skilled professionals in advanced AI research. Ethical concerns, bias in algorithms, and unemployment due to automation are also areas that require careful planning.

Conclusion

India’s role in Artificial Intelligence is growing rapidly. With its large population, strong IT sector, and increasing government support, the country has the potential to become a global leader in AI. By focusing on research, innovation, and social applications, India can ensure that AI becomes a tool for inclusive development and economic growth. The coming decade will be crucial in shaping India’s journey as an AI powerhouse."""

In [33]:
class upscstate(TypedDict):
    essay:str
    language_fbk:str
    analysis_fbk:str
    clarity_fbk:str
    overall_fbk:str
    individual_score:Annotated[list[int],operator.add] #use to add without any conflic all the three value of all all things
    avg_score:float



In [39]:
def evaluate_language(state: upscstate) -> upscstate:
    # build prompt safely with double quotes
    prompt = f"""Evaluate the language quality of the following essay and provide 
    feedback, then assign a score out of 10:

    {state["essay"]}
    """
    
    # invoke model
    output = structured_model.invoke(prompt)

    # assuming output has attributes 'feedback' and 'score'
    return {
        "language_fbk": output.feedback,
        "individual_score": [output.score]
    }


In [40]:
def evaluate_analysis(state:upscstate)->upscstate:
    prompt=f"Evaluate the depth of analysis  of the following essay and provide the feedback and asssign a socre out of 10 \n {state['essay']}"
    output=structured_model.invoke(prompt)
    return {'analysis_fbk':output.feedback,'individual_score':[output.score]}

In [41]:
def evaluate_thoughts(state:upscstate)->upscstate:
    prompt=f"Evaluate the clarity of thought of the following essay and provide the feedback and asssign a socre out of 10 \n {state['essay']}"
    output=structured_model.invoke(prompt)
    return {'clarity_fbk':output.feedback,'individual_score':[output.score]}

In [53]:
def final_evaluation(state: upscstate) -> upscstate:
    # summary 
    prompt = f"""Based on the following feedback, create a summarised feedback:
    Language feedback: {state['language_fbk']}
    Depth of analysis: {state['analysis_fbk']}
    Clarity of thoughts: {state['clarity_fbk']}
    """
    overall_fbk = model.invoke(prompt).content

    # average calculation (fix applied here)
    avg_score = sum(state['individual_score']) / len(state['individual_score'])


    return {
        "overall_fbk": overall_fbk,
        "avg_score": avg_score
    }


In [54]:
graph=StateGraph(upscstate)
graph.add_node("evaluate_language", evaluate_language)
graph.add_node("evaluate_analysis", evaluate_analysis)
graph.add_node("evaluate_thoughts", evaluate_thoughts)
graph.add_node("final_evaluation", final_evaluation)
graph.add_edge(START,"evaluate_language")
graph.add_edge(START,"evaluate_analysis")
graph.add_edge(START,"evaluate_thoughts")

graph.add_edge("evaluate_language","final_evaluation")
graph.add_edge("evaluate_analysis","final_evaluation")
graph.add_edge("evaluate_thoughts","final_evaluation")
graph.add_edge('final_evaluation',END)
workflow=graph.compile()


In [57]:
initial_state={
    'essay':essay2

}
workflow.invoke(initial_state)

{'essay': 'The role of India in AI is very big but also not clear because many people say India is doing good but some say not. In India there are many engineers and also students who are learning computers but still AI is not going fast like America or China. Government sometimes make plan but then plan is not doing good. Some companies are doing AI but mostly it is copy of other country work. The research papers are not high quality and many time it is only for show.\n\nAlso the education system is not giving proper training for AI, many students only learn theory but not practice. This is problem because when they go to job they not know what to do. India say it want to become global leader in AI but infrastructure is weak and internet is not working proper in many places.\n\nIn conclusion, India has many talent but AI growth is slow and confusing and need to become better. Otherwise India will stay behind.',
 'language_fbk': 'The essay addresses the topic of India\'s role in Artifi

In [58]:
essay2="""The role of India in AI is very big but also not clear because many people say India is doing good but some say not. In India there are many engineers and also students who are learning computers but still AI is not going fast like America or China. Government sometimes make plan but then plan is not doing good. Some companies are doing AI but mostly it is copy of other country work. The research papers are not high quality and many time it is only for show.

Also the education system is not giving proper training for AI, many students only learn theory but not practice. This is problem because when they go to job they not know what to do. India say it want to become global leader in AI but infrastructure is weak and internet is not working proper in many places.

In conclusion, India has many talent but AI growth is slow and confusing and need to become better. Otherwise India will stay behind."""